<h1>Bayesian modelling with SIR/SEIR</h1>
<p>We used the covid19_inference library provided at </p>
<p>We rely on pymc3 library to sample from our model</p>

In [ ]:
I#Import the necessary libraries
import os, sys

import cachetools.func

import matplotlib
import matplotlib.colors as colors
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import seaborn as sns
import datetime
import json,urllib
from sklearn.preprocessing import StandardScaler

import pymc3 as pm
import numpy as np
import matplotlib as mpl

sns.set()

<big>The following cells are responsible for crawling the data and processing it</big>

In [ ]:
#-----      
#-----
def fix_region_name(df, pairs = [["Mainland China", "China"]]):
  # fix region names
    for p in pairs:
        df['Country/Region'] = df['Country/Region'].str.replace(p[0],p[1])
    return df

#-----
def merge_df_data(df1,df2):
    return pd.merge(df1, df2,how='left' ,on=['Province/State','Country/Region'])

#-----
def str_add_func(*args):      
    out = []
    for x in args:
        if isinstance(x,str):
            out.append(x)
  
    return '_'.join(out)


class covid_data():
    '''
      Python class to obtain global COVID19 data from 
      John Hopkins GIT repository. This data is updated daily, 
      and the most upto date information available on the web.  
    '''
    def __init__(self,**kwargs):
        nrow = kwargs.get('nrow',None)
        self.confirmed, self.dead, self.recovered = self.get_csseg_data(nrow=nrow)
    @staticmethod
    def create_ts(df):
        ts=df
        columns = ts['region']
        ts=ts.drop(['Province/State', 
                    'Country/Region',
                    'Lat', 
                    'Long',
                    'Population'], 
                   axis=1).set_index('region').T    

        ts.columns = columns 
        ts=ts.fillna(0)
        #
        ts.index.name = 'Date'
        return ts

    def search_agg(self, name,col='Country/Region',ts=True):
    
        if not isinstance(name,list):
            name = [name]

        out = {}
        for k,v in {'confirmed':self.confirmed,
                    'dead':self.dead,
                    'recovered':self.recovered}.items():

        #pd.columns(columns=)
            df_list= []     
            for n in name:
                df = v[v[col]==n].set_index(col).filter(regex='/20')
                df_list.append(df.sum(axis=0))

            df = pd.concat(df_list,axis=1, sort=False)
            df.columns = name
            out[k] = df

        # if ts:                
        #   out[k] = self.create_ts(df)
        # else:
        #   out[k] = df.T

        return out

    def search(self, name,col='Country/Region',ts=True):
        if not isinstance(name,list):
            name = [name]
        out = {}
        for k,v in {'confirmed':self.confirmed,
                    'dead':self.dead,
                    'recovered':self.recovered}.items():
            if ts:                
                out[k] = self.create_ts(v[v[col].map(lambda x: x in name)])
            else:
                out[k] = v[v[col] in name].T
        return out

    @cachetools.func.ttl_cache(maxsize=128, ttl=24 * 60)
    def get_csseg_data(self, nrow=None):
    
        url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master'
        path = f'{url}/csse_covid_19_data/csse_covid_19_time_series' 

        # 
    
        url = f'{path}/time_series_covid19_confirmed_global.csv'
        confirmed = fix_region_name(pd.read_csv(url, nrows=nrow, error_bad_lines=False))
        #
        url = f'{path}/time_series_covid19_deaths_global.csv'
        dead = fix_region_name(pd.read_csv(url, nrows=nrow, error_bad_lines=False))
        #
        url = f'{path}/time_series_covid19_recovered_global.csv'
    
        recovered = fix_region_name(pd.read_csv(url, nrows=nrow, error_bad_lines=False))
        print(confirmed.head())
        #
        return confirmed, dead, recovered


In [ ]:
#Importing the covid_inference library
try:
    import covid19_inference as cov19
except ModuleNotFoundError:
    sys.path.append("../../")
    import covid19_inference as cov19

In [ ]:
cd = covid_data()
cd.confirmed.head()
countries = ['Senegal']
mm = cd.search_agg(countries)

In [ ]:
for ix, ctype in enumerate(['confirmed', 'dead', 'recovered']):
    df = mm[ctype].stack().reset_index()
    #print(df.head())
    df = df.rename(columns={'level_0':'date','level_1':'country',0:ctype})     
    if ix==0:
        df['date'] = pd.to_datetime(df['date'])
        dfall = df
    else:
        dfall[ctype] = df[ctype]

In [ ]:
dfall=dfall.loc[dfall['confirmed']>=100] # ensure we have data with community spread

dfall.drop(['country','recovered'],axis=1,inplace=True)

In [ ]:
#Checks if there are no missing date
assert len(dfall)==(datetime.datetime(2020,8,7)-datetime.datetime(2020,3,26)).days+1 

In [ ]:
#obtain the length of forecast
training_date=datetime.datetime(2020,7,25)
forecast_lenght=(datetime.datetime(2020,8,10)-training_date).days
forecast_lenght

In [ ]:
training_data=dfall.loc[dfall['date']<training_date].set_index(['date'])
validation_data=dfall.loc[dfall['date']>training_date].set_index(['date'])
new_cases=training_data['confirmed']
cum_cases=training_data['confirmed']

In [ ]:
#defines the parameter that is used in modelling
params_model = dict(
    new_cases_obs=new_cases,
    data_begin=datetime.datetime(2020,3,26),
    fcast_len=forecast_lenght,
    diff_data_sim=16,
    N_population=16e6,
)

In [ ]:
#A list of change points- Non pharmceutical intervention
change_points = [
    #religious activities was put on hold
    dict(
        pr_mean_date_transient=datetime.datetime(2020, 3, 10),
        # account for new implementation where transients_day is centered, not begin
        pr_median_transient_len=3,
        pr_sigma_transient_len=0.3,
        pr_sigma_date_transient=3,
        pr_median_lambda=0.2,
        pr_sigma_lambda=0.5,
    ),
    #Travel restrictions
    dict(
        pr_mean_date_transient=datetime.datetime(2020, 3, 15),
        # account for new implementation where transients_day is centered, not begin
        pr_median_transient_len=3,
        pr_sigma_transient_len=0.3,
        pr_sigma_date_transient=3,
        pr_median_lambda=0.2,
        pr_sigma_lambda=0.5,
    ),
    # State of emergency
    dict(
        pr_mean_date_transient=datetime.datetime(2020, 3, 23),
        # account for new implementation where transients_day is centered, not begin
        pr_median_transient_len=3,
        pr_sigma_transient_len=0.3,
        pr_sigma_date_transient=3,
        pr_median_lambda=0.2,
        pr_sigma_lambda=0.5,
    ),
    # strong distancing
    dict(
        pr_mean_date_transient=datetime.datetime(2020, 3, 31)
        + datetime.timedelta(days=1.5),
        pr_median_transient_len=3,
        pr_sigma_transient_len=0.3,
        pr_sigma_date_transient=1,
        pr_median_lambda=1 / 8,
        pr_sigma_lambda=0.5,
    ),
    # the President lifted the state of emergency and curfew
    dict(
        pr_mean_date_transient=datetime.datetime(2020, 6, 30),
        # account for new implementation where transients_day is centered, not begin
        pr_median_transient_len=3,
        pr_sigma_transient_len=0.3,
        pr_sigma_date_transient=3,
        pr_median_lambda=0.2,
        pr_sigma_lambda=0.5,
    ),
    
    
]

In [ ]:
# Median of the prior for the delay in case reporting, we assumed 8 days
pr_delay = 8

In [ ]:
with cov19.Cov19Model(**params_model) as this_model:
    # Create the an array of the time dependent infection rate lambda
    lambda_t_log = cov19.model.lambda_t_with_sigmoids(
        pr_median_lambda_0=0.4,
        pr_sigma_lambda_0=0.5,
        change_points_list=change_points,
        name_lambda_t="lambda_t",  # Name for the variable in the trace
    )

    # Adds the recovery rate mu to the model as a random variable
    mu = pm.Lognormal(name="mu", mu=np.log(1 / 8), sigma=0.2,testval=10)

    # This builds a decorrelated prior for I_begin for faster inference. It is not
    # necessary to use it, one can simply remove it and use the default argument for
    # pr_I_begin in cov19.model.SIR
    prior_I = cov19.model.uncorrelated_prior_I(
        lambda_t_log=lambda_t_log, mu=mu, pr_median_delay=pr_delay
    )

    # Use lambda_t_log and mu as parameters for the SIR model.
    # The SIR model generates the inferred new daily cases.
    new_cases = cov19.model.SIR(lambda_t_log=lambda_t_log, mu=mu, pr_I_begin=prior_I)

    # Delay the cases by a lognormal reporting delay and add them as a trace variable
    new_cases = cov19.model.delay_cases(
        cases=new_cases,
        name_cases="delayed_cases",
        pr_mean_of_median=pr_delay,
        pr_median_of_width=0.1,
    )

    # Modulate the inferred cases by a abs(sin(x)) function, to account for weekend effects
    # Also adds the "new_cases" variable to the trace that has all model features.
    new_cases = cov19.model.week_modulation(cases=new_cases, name_cases="new_cases")

    # Define the likelihood, uses the new_cases_obs set as model parameter
    cov19.model.student_t_likelihood(cases=new_cases)

In [ ]:
trace = pm.sample(model=this_model, tune=2000, draws=2000) #sampling 

In [ ]:
fig, axes = cov19.plot.timeseries_overview(this_model, trace, offset=cum_cases[0])
for ax in axes:
    ax.tick_params(labelrotation=90)

In [ ]:
fig, axes = plt.subplots(6, 3, figsize=(4, 6.4))
axes[0, 2].set_visible(False)
axes[1, 2].set_visible(False)

In [ ]:
# left column
for i, key in enumerate(
    ["weekend_factor", "mu", "lambda_0", "lambda_1", "lambda_2", "lambda_3"]
):
    cov19.plot._distribution(this_model, trace, key, ax=axes[i, 0])

In [ ]:
# mid column
for i, key in enumerate(
    [
        "offset_modulation",
        "sigma_obs",
        "I_begin",
        # beware, these guys were the begin of the transient in the paper,
        # now they are the center points (shifted by transient_len_i)
        "transient_day_1",
        "transient_day_2",
        "transient_day_3",
    ]
):
    cov19.plot._distribution(this_model, trace, key, ax=axes[i, 1])

In [ ]:
# right column
for i, key in enumerate(
    ["delay", "transient_len_1", "transient_len_2", "transient_len_3",]
):
    cov19.plot._distribution(this_model, trace, key, ax=axes[i + 2, 2])

In [ ]:
fig.tight_layout()
fig #To show figure in jupyter notebook